## Erstmal auf den Zyklus gruppieren und dann Features erstellen

offen:
- nur Ref Discharge filtern
- in ein DF
- Zielvariable berechnen (current und RelTime als Features raus) 
- Feature Auswahl mit Beschreibung warum genommen oder eben nicht
- kommentierung

### Import

In [52]:
import pandas as pd
import numpy as np

import os
import sys
import logging

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed\Zyklus'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

In [68]:
def stark_leicht(type_, current_mean, relativeTime_amax):
    entladen_stark = 0
    entladen_leicht = 0
    laden_stark = 0
    laden_leicht = 0
    pause = 0
    
    
    if type_ == 'D': #discharge
        if current_mean >= 3:
            entladen_stark = relativeTime_amax
        else:
            entladen_leicht = relativeTime_amax
    if type_ == 'C': #charge
        if current_mean <= -3:
            laden_stark = relativeTime_amax
        else:
            laden_leicht = relativeTime_amax
    if type_ == 'R':
        pause = relativeTime_amax
    return entladen_stark, entladen_leicht, laden_stark, laden_leicht, pause

    

def temp_hoch(temperature_amax, relativeTime_amax):
    temp_hoch = 0
    if temperature_amax >= 30:
        return relativeTime_amax
    else:
        return 0
        

for file in os.listdir(p_fold):
    if ".csv" in file:
# file = "prepared_RW1.csv" #für tests nur eins
        logging.info(f"<<< Started processing file: {file} >>>")
        new_filename = file.replace("prepared","processed")

        df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                                 #nrows= 100000
                                )
        logging.info(f"<<< Size of Dataframe: {df.shape} >>>")

        #Auf Ebene Batterie + Zyklus gruppieren mit benötigten Aggregationen
        df_pv = pd.pivot_table(
           df,
           index=['batteryname','zyklus','comment','type'],
           aggfunc={'time': np.min, 'relativeTime':  np.max, 'voltage': [np.min, np.max, np.mean], 'current': [np.min, np.max, np.mean], 'temperature': [np.min, np.max, np.mean]}
        ).reset_index()

        df_pv.columns = ['_'.join(col) for col in list(df_pv.columns)]



        df_pv['time_entladen_stark'],df_pv['time_entladen_leicht'],df_pv['time_laden_stark'],df_pv['time_laden_leicht'],df_pv['time_pause']  = zip(*df_pv.apply(lambda x: stark_leicht(x['type_'],x['current_mean'],x['relativeTime_amax']),axis=1))

        df_pv['time_entladen_stark_vorher'] = df_pv.time_entladen_stark.cumsum()
        df_pv['time_entladen_leicht_vorher'] = df_pv.time_entladen_leicht.cumsum()
        df_pv['time_laden_stark_vorher'] = df_pv.time_laden_stark.cumsum()
        df_pv['time_laden_stark_vorher'] = df_pv.time_laden_leicht.cumsum()
        df_pv['time_pause_vorher'] = df_pv.time_pause.cumsum()


        df_pv['time_temp_hoch'] = df_pv.apply(lambda x: temp_hoch(x['temperature_amax'],x['relativeTime_amax']),axis=1)

        df_pv['time_temp_hoch_vorher'] = df_pv.time_temp_hoch.cumsum()

        df_pv['amperestunden'] = (df_pv['current_mean']*df_pv['relativeTime_amax'])/3600
        #mapply(trapz,refDisSteps$relativeTime,refDisSteps$current)/3600

        df_pv_filtered = df_pv[df_pv['comment_'] == 'reference discharge']

        if 

        logging.info(f"<<< Start saving dataframe with shape {df_pv_filtered.shape} >>>")
        df_pv_filtered.to_csv(f"{folder}\\{new_filename}", sep=',', index=False)
        logging.info(f"<<< Saved processed file in directory {folder}\\{new_filename} >>>")

[17:58:13] INFO: <<< Started processing file: prepared_RW1.csv >>>
[17:58:17] INFO: <<< Size of Dataframe: (2958840, 11) >>>
[17:58:19] INFO: <<< Start saving dataframe with shape (48, 27) >>>
[17:58:19] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW1.csv >>>
[17:58:19] INFO: <<< Started processing file: prepared_RW10.csv >>>
[17:58:32] INFO: <<< Size of Dataframe: (8596025, 11) >>>
[17:58:38] INFO: <<< Start saving dataframe with shape (77, 27) >>>
[17:58:38] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW10.csv >>>
[17:58:38] INFO: <<< Started processing file: prepared_RW11.csv >>>
[17:58:53] INFO: <<< Size of Dataframe: (8664510, 11) >>>
[17:58:58] INFO: <<< Start saving dataframe with shape (77, 27) >>>
[17:58:58] INFO: <<<

## DFs zusammenführen

In [91]:
if 'df_fulla' in locals():
    print("a")

In [85]:
d = []

df_full = pd.DataFrame(d, columns=df_pv_filtered.columns)

for file in os.listdir(folder):
    if ".csv" in file:
        df = pd.read_csv(f"{folder}\\{file}")
        df_full = df_full.append(df)
df_full

,batteryname_,zyklus_,comment_,type_,current_amax,current_amin,current_mean,relativeTime_amax,temperature_amax,temperature_amin,...,time_laden_stark,time_laden_leicht,time_pause,time_entladen_stark_vorher,time_entladen_leicht_vorher,time_laden_stark_vorher,time_pause_vorher,time_temp_hoch,time_temp_hoch_vorher,amperestunden
0,RW1,3,reference discharge,D,1.003,0.999,0.999929,7201.82,23.56742,18.39025,...,0,0.0,0.0,0.00,196178.50,194718.28,0.00,0.00,0.000000e+00,2.000364
1,RW1,5,reference discharge,D,1.003,0.999,0.999920,7201.48,24.06639,18.15403,...,0,0.0,0.0,0.00,203379.98,207816.73,0.00,0.00,0.000000e+00,2.000250
2,RW1,1009,reference discharge,D,1.004,0.999,0.999912,7051.15,22.81896,17.57133,...,0,0.0,0.0,17749.62,312669.57,472156.60,16048.39,0.00,1.784186e+05,1.958481
3,RW1,1011,reference discharge,D,1.005,0.999,0.999919,7068.06,23.64538,18.12253,...,0,0.0,0.0,17749.62,319737.63,484111.86,16048.39,0.00,1.784186e+05,1.963192
4,RW1,1925,reference discharge,D,1.004,0.999,0.999907,6881.02,23.89487,17.66582,...,0,0.0,0.0,36155.14,406768.88,743215.33,28472.48,0.00,3.443131e+05,1.911217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,RW9,107540,reference discharge,D,1.002,0.999,0.999919,2826.01,37.88939,31.96866,...,0.0,0.0,0.0,315859.21,4021813.10,4949765.16,1036683.27,2826.01,9.016036e+06,0.784939
76,RW9,110551,reference discharge,D,1.003,0.999,0.999869,2735.55,37.79663,32.10779,...,0.0,0.0,0.0,317601.49,4080113.57,5040620.52,1059934.58,2735.55,9.190943e+06,0.759776
77,RW9,110555,reference discharge,D,1.004,0.999,0.999873,2731.81,37.53384,31.59765,...,0.0,0.0,0.0,317601.49,4082845.38,5060405.85,1067434.58,2731.81,9.220960e+06,0.758740
78,RW9,113572,reference discharge,D,1.003,0.999,0.999889,2695.90,38.29132,32.40151,...,0.0,0.0,0.0,319549.85,4142731.52,5144349.95,1094276.63,2695.90,9.394390e+06,0.748778


,batteryname_,zyklus_,comment_,type_,current_amax,current_amin,current_mean,relativeTime_amax,temperature_amax,temperature_amin,...,time_laden_stark,time_laden_leicht,time_pause,time_entladen_stark_vorher,time_entladen_leicht_vorher,time_laden_stark_vorher,time_pause_vorher,time_temp_hoch,time_temp_hoch_vorher,amperestunden
RW1,NaN,3,reference discharge,D,1.003,0.999,0.999929,7201.82,23.56742,18.39025,...,0,0.0,0.0,0.00,196178.50,194718.28,0.00,0.00,0.000000e+00,2.000364
RW1,NaN,5,reference discharge,D,1.003,0.999,0.999920,7201.48,24.06639,18.15403,...,0,0.0,0.0,0.00,203379.98,207816.73,0.00,0.00,0.000000e+00,2.000250
RW1,NaN,1009,reference discharge,D,1.004,0.999,0.999912,7051.15,22.81896,17.57133,...,0,0.0,0.0,17749.62,312669.57,472156.60,16048.39,0.00,1.784186e+05,1.958481
RW1,NaN,1011,reference discharge,D,1.005,0.999,0.999919,7068.06,23.64538,18.12253,...,0,0.0,0.0,17749.62,319737.63,484111.86,16048.39,0.00,1.784186e+05,1.963192
RW1,NaN,1925,reference discharge,D,1.004,0.999,0.999907,6881.02,23.89487,17.66582,...,0,0.0,0.0,36155.14,406768.88,743215.33,28472.48,0.00,3.443131e+05,1.911217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RW9,NaN,107540,reference discharge,D,1.002,0.999,0.999919,2826.01,37.88939,31.96866,...,0.0,0.0,0.0,315859.21,4021813.10,4949765.16,1036683.27,2826.01,9.016036e+06,0.784939
RW9,NaN,110551,reference discharge,D,1.003,0.999,0.999869,2735.55,37.79663,32.10779,...,0.0,0.0,0.0,317601.49,4080113.57,5040620.52,1059934.58,2735.55,9.190943e+06,0.759776
RW9,NaN,110555,reference discharge,D,1.004,0.999,0.999873,2731.81,37.53384,31.59765,...,0.0,0.0,0.0,317601.49,4082845.38,5060405.85,1067434.58,2731.81,9.220960e+06,0.758740
RW9,NaN,113572,reference discharge,D,1.003,0.999,0.999889,2695.90,38.29132,32.40151,...,0.0,0.0,0.0,319549.85,4142731.52,5144349.95,1094276.63,2695.90,9.394390e+06,0.748778
